In [227]:
# pip install rapidfuzz
# pip install gspread-dataframe
# pip install pandas
# pip install lxml
# pip install tweepy
# pip install gspread

In [228]:
from datetime import datetime, timedelta
import tweepy
import time
import pandas as pd
import random
import gspread
import gspread_dataframe as gd
import rapidfuzz.fuzz as fuzz
import os

In [229]:
def get_all_tweets(id):

    all_tweets = []
    for response in tweepy.Paginator(client.get_users_tweets,
                                    id = f"{id}",
                                    exclude="retweets",
                                    user_fields = ['username', 'public_metrics'],
                                    tweet_fields = ['created_at', 'public_metrics', 'text'],
                                    expansions = ['author_id', 'referenced_tweets.id'],
                                    start_time = (datetime.today() - timedelta(days=10)).isoformat()[0:-7]+"Z",
                                    # end_time = '2022-03-19T00:00:01Z',
                                    max_results=100):
        time.sleep(1)
        all_tweets.append(response)
    
    return all_tweets

In [230]:
def get_all_tweets_info(all_tweets):

    result = []
    user_dict = {}
    # Loop through each response object
    for response in all_tweets:
        # Take all of the users, and put them into a dictionary of dictionaries with the info we want to keep
        for user in response.includes['users']:
            user_dict[user.id] = {'username': user.username, 
                                'followers': user.public_metrics['followers_count'],
                                'tweets': user.public_metrics['tweet_count'],
                                'description': user.description,
                                'location': user.location
                                }
                                        
        for tweet in response.data:
            # For each tweet, find the author's information
            author_info = user_dict[tweet.author_id]

            try:
                reference_status = tweet.referenced_tweets[0].type
            except:
                reference_status = None
            
            # Put all of the information we want to keep in a single dictionary for each tweet
            result.append({'author_id': tweet.author_id, 
                        'username': author_info['username'],
                        'author_followers': author_info['followers'],
                        'author_tweets': author_info['tweets'],
                        'author_description': author_info['description'],
                        'author_location': author_info['location'],
                        'text': tweet.text,
                        'created_at': tweet.created_at,
                        'retweets': tweet.public_metrics['retweet_count'],
                        'replies': tweet.public_metrics['reply_count'],
                        'likes': tweet.public_metrics['like_count'],
                        'quote_count': tweet.public_metrics['quote_count'],
                        'tweet_id': tweet.id,        
                        'reference_status': reference_status
                        })
    
    return result

In [231]:
consumer_key = os.environ['consumer_key']
consumer_secret = os.environ['consumer_secret']
access_token = os.environ['access_token']
access_token_secret = os.environ['access_token_secret']
bearer_token = os.environ['bearer_token']

In [232]:
client = tweepy.Client(bearer_token=bearer_token, consumer_key=consumer_key, consumer_secret=consumer_secret, access_token=access_token, access_token_secret=access_token_secret, wait_on_rate_limit=True)

In [233]:
gc = gspread.service_account(filename='creds.json')
ws = gc.open("trackfounders").worksheet("Followings")
old_followings = gd.get_as_dataframe(ws)
old_followings = old_followings.dropna(axis=1, how='all')
old_followings

,ShivAroor,ShekharGupta,captriturathee,ashutosh83B,kunalb11,supriyapaul93,deepigoyal,viraj_sheth,bhash,vijayshekhar,...,rohitkbansal,Chikisarkar,ashishkashyap,kavinbm,harshilmathur,telljeeves,AnshulSushil,Aakriti,GabbbarSingh,Being_Humor
0,manujosephsan,ELuttwak,jsaideepak,AnupamConnects,stephsmithio,adadithya,IRONMANtri,_buildd,CNBCTV18Live,maheshperi,...,naval,bridgerton,AyushkWadhwa,sayinshallah,zacliew,chennamaneni,yamini_bhat,pheadrick,Being_Humor,LegalTalwar
1,joe_sameer,carlbildt,SadiqAl55350795,rajinikanth,shitPM,BeriArpit,deepdbhandari,vandana_menon,CNBCTV18News,NeinQuarterly,...,SnapdealGujarat,orangeturban,duatweets,gregjoz,stevekucia,shantanukd,anusehgal,KhadimBatti,vasudev_ravi,SouravArts
2,KumarKunalmedia,IAPonomarenko,Neeraj_chopra1,SenseandC_sense,DmytroKuleba,abhisheknag,AnilSinghvi_,satyan,AionicsInc,TeamBlind,...,fafsters,Karlwheel,GitaGopinath,_Kaspar__,Tejasvi_Surya,rahulrmv,anupohaney,AnishaPatnaik,JohnHulsman1,kaykaymenon02
3,AxisMyIndia,BinaShah,iamsrk,samajwadiparty,SahilBloom,pratshukla,UltrahumanHQ,PoddarNisha,yurimilner,gauravg85,...,mrgirish,ptrmadurai,anuhyaprayaga,Fwiz,kiranshaw,DeepikaDakuda,jagora,AiyyoShraddha,ClintEhrlich,MattLaemon
4,ItsShubhangi,KyivIndependent,ShishirGoUP,puru_ag,McConaughey,MohapatraHemant,vatsalsinghal,Ashneer_Grover,harari_yuval,NetaFlixIndia,...,geneliad,devangshudatta,udaykotak,AdamMGrant,thetanmay,ManishaRaisingh,Priyamouli1812,bhavintu,W7VOA,GabbbarSingh
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3930,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3931,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3932,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3933,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [234]:
# old_followings["ShivAroor"].astype(str)

In [235]:
ws = gc.open("trackfounders").worksheet("Designation")
designation = gd.get_as_dataframe(ws)
designation = designation.dropna(axis=1, how='all')
designation = designation.dropna(axis=0, how='all')
designation.drop_duplicates(subset = ['usernames'], inplace = True)

In [236]:
usernames_designation = designation.set_index("usernames")['designation'].to_dict()
usernames_designation

{'ShivAroor': 'Founder @Livefist',
 'ShekharGupta': 'Founder ThePrint',
 'captriturathee': "Co-founder of 'Flying Beast'",
 'ashutosh83B': 'Co-founder - http://satyahindi.com',
 'kunalb11': 'Founder - @CRED_club',
 'supriyapaul93': 'Co-Founder & CEO @JoshTalksLive @joshskillsapp',
 'deepigoyal': 'Founder - @zomato',
 'viraj_sheth': 'Co-founder @MonkEtweets',
 'bhash': 'Founder @Olacabs',
 'vijayshekhar': 'Founder @Paytm',
 'gargashutosh': 'Founder - The Brand Called You, Guardian Pharmacy',
 'palakzat': 'Founder - Sublist',
 'gauravmunjal': 'Founder - Unacademy Group',
 'TheQtiyapaGuy': 'Founder - TVF (TheViralFever)',
 'SandeepAgg': 'Founder @letsdroom',
 'riteshagar': 'Founder & Group CEO @oyorooms',
 'warikoo': 'Entrepreneur, founder http://nearbuy.com',
 'NaveenTewari': 'Founder - InMobi, Glance, Roposo',
 'paraschopra': 'Founder @wingify',
 'deepakabbot': 'Co-founder at http://indiagold.co',
 'alokebajpai': 'Founder @ixigo',
 'Rahul_J_Mathur': 'Founder & CEO - @VerakInsurance',
 '

In [237]:
ws = gc.open("trackfounders").worksheet("Description")
old_description = gd.get_as_dataframe(ws)
old_description = old_description.dropna(axis=1, how='all')
old_description = old_description.dropna(axis=0, how='all')
old_description['description'] = old_description['description'].fillna("")

In [192]:
usernames = list(designation['usernames'])

new_followings = pd.DataFrame()
all_description = []
all_description_usernames = []
id_username = {}

for i in usernames:

    try:
        user = client.get_user(username = i, user_fields = ['description'])
    except Exception as e:
        # print(e)
        continue
    
    id_username[user.data.id] = user.data.username
    
    all_following_data = []

    for response in tweepy.Paginator(client.get_users_following, 
                                    user_fields = ['username', 'description'],
                                    id = user.data.id,
                                    max_results=1000):
        all_following_data.append(response)
        time.sleep(60)

    all_following_usernames = []

    try:  
        for k in all_following_data:
            for l in k.data:
                all_following_usernames.append(l.username)
    except Exception as e:
        # print(e)
        # print(all_following_data)
        continue

    all_description.append(user.data.description)
    all_description_usernames.append(i)

    new_followings = pd.merge(new_followings, pd.DataFrame(all_following_usernames, columns=[i]), left_index=True, right_index=True, how='outer')

In [193]:
new_description = pd.DataFrame({"usernames": all_description_usernames, "description": all_description})
new_description['description'] = new_description['description'].fillna("")

In [194]:
gc = gspread.service_account(filename='creds.json')
ws = gc.open("trackfounders").worksheet("Posting Date")
posting_date = gd.get_as_dataframe(ws)
posting_date = posting_date.dropna(axis=1, how='all')
posting_date = posting_date.dropna(axis=0, how='all')
posting_date['day'] = posting_date['day'].astype(int)
posting_date.drop_duplicates(subset = ['usernames'], inplace = True)

In [195]:
usernames_posting_date = posting_date.set_index("usernames")['day'].to_dict()

In [196]:
all_tweets_engagement = []
all_engagement_score = {}

for user_ids in list(id_username.keys()):

    username_eng = id_username.get(user_ids)

    if usernames_posting_date.get(username_eng) == datetime.today().weekday():

        all_tweets = get_all_tweets(user_ids)
        
        if all_tweets[0].data is not None:

            result = get_all_tweets_info(all_tweets)

            for data in result:
                data['engagement'] = data['likes'] + data['replies'] + data['retweets'] + data['quote_count']
            
            df = pd.DataFrame(result)

            if df['engagement'].max() > 20:
                
                df_max_engagement = df[df['engagement'] == df['engagement'].max()]
                
                most_eng_tweet_ids = list(df_max_engagement['tweet_id'])

                most_eng_tweet_username = list(df_max_engagement['username'])

                for k,j in zip(most_eng_tweet_ids, most_eng_tweet_username):
                    all_tweets_engagement.append({j:k})
                    all_engagement_score[j] = df['engagement'].max()

            # for k,j in zip(most_eng_tweet_ids, most_eng_tweet_username):
            #     all_tweets_engagement.append({j:k})            

In [244]:
# New Followings

for i in old_followings.columns:
    if i in new_followings.columns:
        for j in [x for x in list(set(new_followings[i].astype(str)) - set(old_followings[i].astype(str))) if x == x]:
            if usernames_designation.get(i) is None:
                # client.create_tweet(text = f"✅@{i} is now following @{j}")
                print(f"✅@{i} is now following @{j}")
            else:
                # client.create_tweet(text = f"✅@{i} ({usernames_designation.get(i)}) is now following @{j}")
                print(f"✅@{i} ({usernames_designation.get(i)}) is now following @{j}")
            # client.create_tweet(text = f"@{i} is now following @{j}")
            time.sleep(random.randint(3, 10))

In [198]:
# Most Engaged

for i in all_tweets_engagement:
    tweet_url = f"https://twitter.com/{list(i.keys())[0]}/status/{str(list(i.values())[0])}"
    username = list(i.keys())[0]
    client.create_tweet(text = f"🏅 Most Enagaged Tweet of @{username} ({usernames_designation.get(username)}) in the last 10 days:\n\nTotal Engagement Score: {all_engagement_score.get(username)}\n\n{tweet_url}")
    time.sleep(10)

In [199]:
# for i in all_tweets_engagement:
#     tweet_url = f"https://twitter.com/{list(i.values())[0]}/status/{str(list(i.keys())[0])}"
#     username = {list(i.values())[0]}
#     print(tweet_url, username)

In [200]:
# for i in all_tweets_engagement:
#     print(list(i.values())[0])

In [201]:
# # New Unollowings

# for i in old_followings.columns:
#     if i in new_followings.columns:
#         for j in [x for x in list(set(old_followings[i]) - set(new_followings[i])) if x == x]:
#             if usernames_designation.get(i) is None:
#                 client.create_tweet(text = f"💔@{i} is no longer following @{j}")
#                 # print(f"💔@{i} is no longer following @{j}")
#             else:
#                 client.create_tweet(text = f"💔@{i} ({usernames_designation.get(i)}) is no longer following @{j}")
#                 # print(f"💔@{i} ({usernames_designation.get(i)}) is no longer following @{j}")
#             # client.create_tweet(text = f"@{i} is now following @{j}")
#             time.sleep(random.randint(3, 10))

In [202]:
compare_description = pd.merge(old_description, new_description, on = "usernames")
compare_description.columns = ['usernames', 'old_description', 'new_description']
compare_description['ratio'] = compare_description.apply(lambda x: fuzz.ratio(str(x['old_description'].lower()),str(x['new_description'].lower())), axis = 1)
compare_description['ratio'] = compare_description['ratio'].astype(int)

In [203]:
for i, j, k in zip(compare_description['usernames'], compare_description['new_description'], compare_description['ratio']):
    if k < 90:
        client.create_tweet(text = f"Updated bio of @{i}: {j}")
        time.sleep(random.randint(3, 10))

In [204]:
ws = gc.open("trackfounders").worksheet("Followings")
ws.clear()
gd.set_with_dataframe(ws, new_followings)
time.sleep(random.randint(5,10))

In [205]:
ws = gc.open("trackfounders").worksheet("Description")
ws.clear()
gd.set_with_dataframe(ws, new_description)
time.sleep(random.randint(5,10))